# Загрузка Pandas и очистка данных

In [1]:
from collections import Counter
import pandas as pd
import re
import datetime

In [2]:
df = pd.read_csv('main_task.csv')
df.head(3)

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781


## Функции

In [3]:
def price_range(text):
    # create price range in digits
    if text == "$$ - $$$":
        return 100
    elif text == "$$$$":
        return 1000
    elif text == "$":
        return 1
    else:
        return None

In [4]:
def del_not_digit_data(dataframe):
    for column in dataframe.columns:
        try:
            float(dataframe[column][0])
        except Exception as ex:
            print(ex)
            dataframe.drop([column], axis=1, inplace=True)
    return dataframe

In [5]:
df['Price Range'].value_counts()

$$ - $$$    18412
$            6279
$$$$         1423
Name: Price Range, dtype: int64

In [6]:
# Ваш код по очистке данных и генерации новых признаков
# При необходимости добавьте ячейки
# df['Price Range'] = df['Price Range'].apply(price_range)
# df['Price Range'].fillna(df['Price Range'].mean(), inplace=True)
# df['Price Range'].value_counts()

In [7]:
df['Price Range'] = df['Price Range'].apply(price_range)
prices = pd.get_dummies(df['Price Range'])
df = df.drop('Price Range', axis=1)
df = df.join(prices)

In [8]:
cities = pd.get_dummies(df.City)
df = df.drop('City', axis=1)
df = df.join(cities)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 42 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   Cuisine Style      30717 non-null  object 
 2   Ranking            40000 non-null  float64
 3   Rating             40000 non-null  float64
 4   Number of Reviews  37457 non-null  float64
 5   Reviews            40000 non-null  object 
 6   URL_TA             40000 non-null  object 
 7   ID_TA              40000 non-null  object 
 8   1.0                40000 non-null  uint8  
 9   100.0              40000 non-null  uint8  
 10  1000.0             40000 non-null  uint8  
 11  Amsterdam          40000 non-null  uint8  
 12  Athens             40000 non-null  uint8  
 13  Barcelona          40000 non-null  uint8  
 14  Berlin             40000 non-null  uint8  
 15  Bratislava         40000 non-null  uint8  
 16  Brussels           400

In [9]:
pattern = re.compile('\d+/\d+/\d+')
df['date_of_Review'] = df['Reviews'].apply(lambda x: re.findall(pattern, x))
df['data_2'] = df['date_of_Review'].apply(lambda x: pd.to_datetime(x).max())
df['data_1'] = df['date_of_Review'].apply(lambda x: pd.to_datetime(x).min())

[['Good food at your doorstep', 'A good hotel restaurant'], ['12/31/2017', '11/20/2017']]


In [10]:
print(df.data_2.max())
df["date_diff"] = df.data_2 - df.data_1
df["date_diff"] = df.date_diff.apply(lambda x: float(x.days))
print(df.date_diff.max())

2018-02-26 00:00:00
3207.0


In [11]:
cuisine = Counter()
cuisines = set()
cuisines_variance = []
pattern = re.compile("[a-zA-Z]+\s*[a-zA-Z]*\s*[a-zA-Z]*\s*[a-zA-Z]*\s*[a-zA-Z]*\s*[a-zA-Z]*\s*[a-zA-Z]*")
test = []
for x in df['Cuisine Style']:
    try:
        epic = re.findall(pattern, x)
        for i in epic:
            cuisines.add(i)
        test.append(epic)
    except:
        test.append(['pass'])
        pass
print(test[0])
for cell in test:
    try:
        # cell = cell.strip("[,]")
        # cell = cell.split()
        # print(cell)
        cuisines_variance.append(len(cell))
        for i in cell:
            cuisine[i] += 1
    except:
        cuisines_variance.append(1)
        pass
df["cuisine_variance"] = cuisines_variance
# print(sorted(cuisines))
# print(len(cuisines))
# print(cuisine.most_common(1))
# print(sum(cuisines_variance)/len(cuisines_variance))

['European', 'French', 'International']


In [12]:
df = del_not_digit_data(df)
df.info()

could not convert string to float: 'id_5569'
could not convert string to float: "['European', 'French', 'International']"
could not convert string to float: "[['Good food at your doorstep', 'A good hotel restaurant'], ['12/31/2017', '11/20/2017']]"
could not convert string to float: '/Restaurant_Review-g187147-d1912643-Reviews-R_Yves-Paris_Ile_de_France.html'
could not convert string to float: 'd1912643'
float() argument must be a string or a number, not 'list'
float() argument must be a string or a number, not 'Timestamp'
float() argument must be a string or a number, not 'Timestamp'
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 39 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Ranking            40000 non-null  float64
 1   Rating             40000 non-null  float64
 2   Number of Reviews  37457 non-null  float64
 3   1.0                40000 non-null  uint8  
 4   100.0 

In [13]:
df.fillna(df['Number of Reviews'].min(), inplace=True)
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 39 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Ranking            40000 non-null  float64
 1   Rating             40000 non-null  float64
 2   Number of Reviews  40000 non-null  float64
 3   1.0                40000 non-null  uint8  
 4   100.0              40000 non-null  uint8  
 5   1000.0             40000 non-null  uint8  
 6   Amsterdam          40000 non-null  uint8  
 7   Athens             40000 non-null  uint8  
 8   Barcelona          40000 non-null  uint8  
 9   Berlin             40000 non-null  uint8  
 10  Bratislava         40000 non-null  uint8  
 11  Brussels           40000 non-null  uint8  
 12  Budapest           40000 non-null  uint8  
 13  Copenhagen         40000 non-null  uint8  
 14  Dublin             40000 non-null  uint8  
 15  Edinburgh          40000 non-null  uint8  
 16  Geneva             400

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [14]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['Rating'], axis = 1)
y = df['Rating']

In [15]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [16]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [17]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [18]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [19]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.214691
